In [70]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext("local", "FantaStats")
spark = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=FantaStats, master=local) created by __init__ at <ipython-input-1-7ae76705822f>:4 

In [84]:
from pyspark.sql.functions import *

def loadCSV(sdf, filename):
    df = sdf.load(filename)
    return df.withColumn("Gi", regexp_replace("Gi", " ", "-"))
    
def prefix(sdf, prefix):
      for c in sdf.columns:
          sdf = sdf.withColumnRenamed(c, '{}{}'.format(prefix, c))
      return sdf

In [86]:
csvContext = spark.read.format("csv").option("header", "true").option("delimiter", ";")

players_2015 = prefix(loadCSV(csvContext, "2015-2016.csv"), "_2015_")
players_2016 = prefix(loadCSV(csvContext, "2016-2017.csv"), "_2016_")
players_2017 = prefix(loadCSV(csvContext, "2017-2018.csv"), "_2017_")
players_2018 = prefix(loadCSV(csvContext, "2018-2019.csv"), "_2018_")
current_players = prefix(loadCSV(csvContext, "2019-2020.csv"), "c_")

In [92]:
current_2015 = current_players.join(players_2015, current_players.c_Gi == players_2015._2015_Gi, how='left')
current_2015_2016 = current_2015.join(players_2016, current_2015.c_Gi == players_2016._2016_Gi, how='left')
current_2015_2016_2017 = current_2015_2016.join(players_2017, current_2015_2016.c_Gi == players_2017._2017_Gi, how='left')
current_2015_2016_2017_2018 = current_2015_2016_2017.join(players_2018, current_2015_2016_2017.c_Gi == players_2018._2018_Gi, how='left')

In [117]:
columns_to_drop = ["_2015_Gi", "_2016_Gi", "_2017_Gi", "_2018_Gi",
                  "_2015_S", "_2016_S", "_2017_S", "_2018_S",
                  "_2015_Au", "_2016_Au", "_2017_Au", "_2018_Au",
                  "_2015_R", "_2016_R", "_2017_R", "_2018_R",
                  "_2015_IAM", "_2016_IAM", "_2017_IAM", "_2018_IAM", ]
finalRoster = current_2015_2016_2017_2018.drop(*columns_to_drop)

In [120]:
columns_to_drop_keepers = ["_2015_As", "_2016_As", "_2017_As", "_2018_As",
                           "_2015_RS", "_2016_RS", "_2017_RS", "_2018_RS", "c_R"]
keepers = finalRoster.filter("_2018_R == 'P'").drop(*columns_to_drop_keepers)
display(keepers.toPandas())

,c_S,c_Gi,c_Q,_2015_P,_2015_MV,_2015_FMV,_2015_G,_2015_Am,_2015_Es,_2015_Gs,...,_2017_Gs,_2017_RP,_2018_P,_2018_MV,_2018_FMV,_2018_G,_2018_Am,_2018_Es,_2018_Gs,_2018_RP
0,INTER,HANDANOVIC-Samir,20,36,6.39,5.46,0,5,0,34,...,30,0,37,6.3,5.42,0,0,0,33,0
1,JUVENTUS,SZCZESNY-Wojciech,19,34,6.21,5.19,0,1,0,34,...,9,0,25,6.18,5.55,0,0,0,20,1
2,MILAN,DONNARUMMA-Gianluigi,18,30,6.25,5.23,0,3,0,29,...,42,0,34,6.32,5.49,0,4,0,30,1
3,NAPOLI,MERET-Alex,17,None,None,None,None,None,None,None,...,16,2,14,6.18,5.46,0,0,1,9,0
4,TORINO,SIRIGU-Salvatore,17,None,None,None,None,None,None,None,...,44,3,36,6.49,5.67,0,1,0,35,2
5,LAZIO,STRAKOSHA-Thomas,15,None,None,None,None,None,None,None,...,49,1,34,6.09,4.91,0,4,0,39,0
6,ATALANTA,GOLLINI-Pierluigi,15,26,6.21,4.48,0,2,0,44,...,4,0,20,6.18,5.1,0,1,0,21,0
7,CAGLIARI,CRAGNO-Alessio,14,None,None,None,None,None,None,None,...,45,2,38,6.53,5.25,0,1,0,54,2
8,FIORENTINA,DRAGOWSKI-Bartolomiej,13,None,None,None,None,None,None,None,...,8,0,16,6.5,5.12,0,2,0,25,1
9,BOLOGNA,SKORUPSKI-Lukasz,12,31,6.11,4.69,0,2,0,43,...,0,0,38,6.07,4.58,0,1,0,56,0
